In [27]:
import org.apache.spark.sql.SparkSession

In [28]:
val spark = SparkSession.
    builder.
    config("spark.ui.port", "0").
    config("spark.sql.warehouse.dir", "/user/itv001389/warehouse").
    enableHiveSupport.
    master("yarn").
    appName("Getting Started - Spark with Scala").
    getOrCreate

spark = org.apache.spark.sql.SparkSession@4fad5511


org.apache.spark.sql.SparkSession@4fad5511

In [29]:
import spark.implicits._
import spark.sql

In [30]:
spark.sqlContext.setConf("hive.exec.max.dynamic.partitions.pernode" , "100000")

In [34]:
spark.sqlContext.setConf("hive.exec.max.dynamic.partitions","100000")

In [35]:
spark.sql("use Raj")

[]

In [36]:
spark.sql("show tables").show(false)

+--------+------------------+-----------+
|database|tableName         |isTemporary|
+--------+------------------+-----------+
|raj     |order_immu        |false      |
|raj     |orderitem_nor     |false      |
|raj     |orderitem_orc     |false      |
|raj     |orderitem_par     |false      |
|raj     |order_items       |false      |
|raj     |order_items_part  |false      |
|raj     |order_oi          |false      |
|raj     |order_oi_cast     |false      |
|raj     |order_oi_cast1    |false      |
|raj     |order_oi_part     |false      |
|raj     |order_oi_part1    |false      |
|raj     |order_oi_part2    |false      |
|raj     |order_oi_partit   |false      |
|raj     |order_oi_partition|false      |
|raj     |order_oi_sum1     |false      |
|raj     |order_oi_sum2     |false      |
|raj     |order_orc         |false      |
|raj     |orders            |false      |
|raj     |order_skip_header |false      |
|raj     |order_skip_header1|false      |
+--------+------------------+-----

In [5]:
spark.sql("describe formatted orders").show(false)

+----------------------------+--------------------------------------------------------------------+-------+
|col_name                    |data_type                                                           |comment|
+----------------------------+--------------------------------------------------------------------+-------+
|order_id                    |int                                                                 |null   |
|Order_date                  |timestamp                                                           |null   |
|Order_product_id            |int                                                                 |null   |
|Order_status                |string                                                              |null   |
|                            |                                                                    |       |
|# Detailed Table Information|                                                                    |       |
|Database                   

##### The division of table into related parts based on some columns is Partitioning.
#####  Partitions are created on columns on which we use filter conditions multiple times.
#####  There are two types of Partitions 1.Static Partition 2.Dynamic partition
#####  In static partition method we need to manually specify the partitions.This will bluntly load
#####  whereas in dynamic method we can dynamically partition the table


##### Horizontal division of table takes place in partition.
#####  We generally create partitions are the columns whose cardinality value is less and also whose column is mostly used in WHERE clause in SQL
#####  If we query partition table then query will directly hit that partition and retrive data instead of scanning entire table
#####  There are two types of partitions 1) Static 2) Dynamic
#####  Static partition ---> we need to manually define the partition value
##### There are two methods in Static partition 1) Load method 2) Insert select method
##### Load method will directly load the data without checking column order whereas in insert select we can have control which column should go where
##### Dynamic partition ---> In this partitions are created dynamically based on the unique values in that column.This gives more flexibily as we no need to specify partition values manully
##### Dynamic partition also have two methods  1) Load method 2) Insert select method
##### Load method will directly load the data without checking column order whereas in insert select we can have control which column should go where
##### In order to have dynamic partition enabled we need to use below mentioned two properties

##### set hive.exec.dynamic.partition=true; 
##### set hive.exec.dynamic.partition.mode=nonstrict;

In [9]:
spark.sql("create table Raj.orders_part(order_id int,order_date timestamp,order_product_id int) partitioned by (order_status string) ")

lastException: Throwable = null


[]

#####   Property should be added for dynamic partition

In [8]:
spark.sql("set hive.exec.dynamic.partition.mode=nonstrict")

lastException: Throwable = null


[key: string, value: string]

In [11]:
spark.sql("insert into table Raj.orders_part select * from orders")

[]

In [12]:
spark.sql("select * from Raj.orders_part").show(false)

+--------+-------------------+----------------+------------+
|order_id|order_date         |order_product_id|order_status|
+--------+-------------------+----------------+------------+
|21      |2013-07-25 00:00:00|2711            |PENDING     |
|36      |2013-07-25 00:00:00|5649            |PENDING     |
|39      |2013-07-25 00:00:00|8214            |PENDING     |
|42      |2013-07-25 00:00:00|9776            |PENDING     |
|44      |2013-07-25 00:00:00|10500           |PENDING     |
|49      |2013-07-25 00:00:00|1871            |PENDING     |
|55      |2013-07-25 00:00:00|2052            |PENDING     |
|68      |2013-07-25 00:00:00|4320            |PENDING     |
|85      |2013-07-25 00:00:00|1485            |PENDING     |
|96      |2013-07-25 00:00:00|8683            |PENDING     |
|97      |2013-07-25 00:00:00|10784           |PENDING     |
|121     |2013-07-26 00:00:00|2074            |PENDING     |
|132     |2013-07-26 00:00:00|289             |PENDING     |
|158     |2013-07-26 00:

In [13]:
spark.sql("create table Raj.order_Items_part(order_item_id int,order_item_order_id int,order_item_product_id int,order_item_subtotal decimal(17,4),order_item_product_price decimal(17,4)) partitioned by(order_item_quantity int)")

[]

In [15]:
spark.sql("insert into table Raj.order_Items_part select * from order_items ")

++
||
++
++



In [16]:
spark.sql("select * from Raj.order_Items_part ").show(false)

+-------------+-------------------+---------------------+-------------------+------------------------+-------------------+
|order_item_id|order_item_order_id|order_item_product_id|order_item_subtotal|order_item_product_price|order_item_quantity|
+-------------+-------------------+---------------------+-------------------+------------------------+-------------------+
|144765       |57861              |743                  |1.0000             |169.9900                |169                |
|145535       |58164              |743                  |1.0000             |169.9900                |169                |
|145554       |58170              |743                  |1.0000             |169.9900                |169                |
|145698       |58232              |743                  |1.0000             |169.9900                |169                |
|146153       |58395              |743                  |1.0000             |169.9900                |169                |
|146591       |5

In [17]:
spark.sql("describe formatted Raj.order_Items_part").show(false)

+----------------------------+----------------------------------+-------+
|col_name                    |data_type                         |comment|
+----------------------------+----------------------------------+-------+
|order_item_id               |int                               |null   |
|order_item_order_id         |int                               |null   |
|order_item_product_id       |int                               |null   |
|order_item_subtotal         |decimal(17,4)                     |null   |
|order_item_product_price    |decimal(17,4)                     |null   |
|order_item_quantity         |int                               |null   |
|# Partition Information     |                                  |       |
|# col_name                  |data_type                         |comment|
|order_item_quantity         |int                               |null   |
|                            |                                  |       |
|# Detailed Table Information|        

In [18]:
spark.sql("show tables").show(false)

+--------+------------------+-----------+
|database|tableName         |isTemporary|
+--------+------------------+-----------+
|raj     |order_immu        |false      |
|raj     |orderitem_nor     |false      |
|raj     |orderitem_orc     |false      |
|raj     |orderitem_par     |false      |
|raj     |order_items       |false      |
|raj     |order_items_part  |false      |
|raj     |order_oi          |false      |
|raj     |order_oi_cast     |false      |
|raj     |order_oi_cast1    |false      |
|raj     |order_oi_sum1     |false      |
|raj     |order_oi_sum2     |false      |
|raj     |order_orc         |false      |
|raj     |orders            |false      |
|raj     |order_skip_header |false      |
|raj     |order_skip_header1|false      |
|raj     |order_skip_header2|false      |
|raj     |order_skip_header3|false      |
|raj     |order_snap        |false      |
|raj     |orders_part       |false      |
+--------+------------------+-----------+



In [23]:
spark.sql("select * from order_oi").show(10)

+--------+-------------------+---------------+-------------+-------------------+------------------------+
|order_id|         order_date|   order_status|order_item_id|order_item_quantity|order_item_product_price|
+--------+-------------------+---------------+-------------+-------------------+------------------------+
|       1|2013-07-25 00:00:00|         CLOSED|            1|                  1|                  1.0000|
|       2|2013-07-25 00:00:00|PENDING_PAYMENT|            2|                  1|                  1.0000|
|       2|2013-07-25 00:00:00|PENDING_PAYMENT|            3|                  5|                  5.0000|
|       2|2013-07-25 00:00:00|PENDING_PAYMENT|            4|                  1|                  1.0000|
|       4|2013-07-25 00:00:00|         CLOSED|            5|                  2|                  2.0000|
|       4|2013-07-25 00:00:00|         CLOSED|            6|                  5|                  5.0000|
|       4|2013-07-25 00:00:00|         CLOSED|

#####   Before creating partitions on multiple tables first need to join the tables into a single table

In [48]:
spark.sql("create table Raj.order_oi_partit(order_id int,order_item_id int,order_item_quantity int,order_item_product_price decimal(17,4),order_date timestamp,order_status string) ")

lastException: Throwable = null


[]

#####   while creating partitions order of the columns are important as they should be in an order else data will get shuffeled

In [52]:
spark.sql("insert into table Raj.order_oi_partit select a.order_id,b.order_item_id,b.order_item_quantity,b.order_item_quantity,a.order_date,a.order_status from orders a join order_items b on (a.order_id=b.order_item_order_id)")

lastException: Throwable = null


[]

In [53]:
spark.sql("select * from Raj.order_oi_partit").show(5)

+--------+-------------+-------------------+------------------------+-------------------+---------------+
|order_id|order_item_id|order_item_quantity|order_item_product_price|         order_date|   order_status|
+--------+-------------+-------------------+------------------------+-------------------+---------------+
|       1|            1|                  1|                  1.0000|2013-07-25 00:00:00|         CLOSED|
|       2|            2|                  1|                  1.0000|2013-07-25 00:00:00|PENDING_PAYMENT|
|       2|            3|                  5|                  5.0000|2013-07-25 00:00:00|PENDING_PAYMENT|
|       2|            4|                  1|                  1.0000|2013-07-25 00:00:00|PENDING_PAYMENT|
|       4|            5|                  2|                  2.0000|2013-07-25 00:00:00|         CLOSED|
+--------+-------------+-------------------+------------------------+-------------------+---------------+
only showing top 5 rows



In [55]:
spark.sql("create table Raj.order_oi_partition(order_id int,order_item_id int,order_item_quantity int,order_item_product_price decimal(17,4)) partitioned by (order_date timestamp,order_status string)")

lastException: Throwable = null


[]

In [5]:
spark.sql("show tables").show(false)

+--------+------------------+-----------+
|database|tableName         |isTemporary|
+--------+------------------+-----------+
|raj     |order_immu        |false      |
|raj     |orderitem_nor     |false      |
|raj     |orderitem_orc     |false      |
|raj     |orderitem_par     |false      |
|raj     |order_items       |false      |
|raj     |order_items_part  |false      |
|raj     |order_oi          |false      |
|raj     |order_oi_cast     |false      |
|raj     |order_oi_cast1    |false      |
|raj     |order_oi_part     |false      |
|raj     |order_oi_part1    |false      |
|raj     |order_oi_part2    |false      |
|raj     |order_oi_partit   |false      |
|raj     |order_oi_partition|false      |
|raj     |order_oi_sum1     |false      |
|raj     |order_oi_sum2     |false      |
|raj     |order_orc         |false      |
|raj     |orders            |false      |
|raj     |order_skip_header |false      |
|raj     |order_skip_header1|false      |
+--------+------------------+-----

In [5]:
spark.sql("SET hive.exec.max.dynamic.partitions=100000")

[key: string, value: string]

In [6]:
spark.sql("SET hive.exec.max.dynamic.partitions.pernode=100000")

[key: string, value: string]

In [ ]:
spark.sql("insert into table Raj.order_oi_partition select * from Raj.order_oi_partit")